## Partie 1 : POS Tagging Médical

In [66]:
import spacy
import re

texte = "La patiente âgée de 65 ans présente une toux persistante et une fièvre à 38.5°C. Le médecin prescrit de l'amoxicilline 500mg 3x/jour."
nlp = spacy.load("fr_core_news_md")


In [67]:
pattern = re.compile(r"(\d+x/jour|\d+(?:\.\d+)?°C|\d+mg|\d+g)")

# Personnaliser le tokenizer avec token_match
def custom_tokenizer(nlp):
    return spacy.tokenizer.Tokenizer(
        nlp.vocab,
        token_match=pattern.match,
        rules=nlp.Defaults.tokenizer_exceptions,
        prefix_search=nlp.tokenizer.prefix_search,
        suffix_search=nlp.tokenizer.suffix_search,
        infix_finditer=nlp.tokenizer.infix_finditer
    )
nlp.tokenizer = custom_tokenizer(nlp)
doc = nlp(texte)

In [68]:
for token in doc:
    print(
        f"Mot: {token.text:<15}",
        f"POS: {token.pos_:<8}",
        f"Lemme: {token.lemma_:<15}",
        f"Dépendance: {token.dep_:<12}",
        f"Tête: {token.head.text:<12}"
    )

Mot: La              POS: DET      Lemme: le              Dépendance: det          Tête: patiente    
Mot: patiente        POS: NOUN     Lemme: patient         Dépendance: nsubj        Tête: présente    
Mot: âgée            POS: ADJ      Lemme: âgé             Dépendance: acl          Tête: patiente    
Mot: de              POS: ADP      Lemme: de              Dépendance: case         Tête: ans         
Mot: 65              POS: NUM      Lemme: 65              Dépendance: nummod       Tête: ans         
Mot: ans             POS: NOUN     Lemme: an              Dépendance: obl:arg      Tête: âgée        
Mot: présente        POS: VERB     Lemme: présente        Dépendance: ROOT         Tête: présente    
Mot: une             POS: DET      Lemme: un              Dépendance: det          Tête: persistante 
Mot: toux            POS: NOUN     Lemme: toux            Dépendance: amod         Tête: persistante 
Mot: persistante     POS: NOUN     Lemme: persistante     Dépendance: obj         

In [69]:
symptomes = []
for token in doc:
    if token.dep_ == "obj" and token.head.lemma_ == "présente":
        mods = [child.text for child in token.children if child.dep_ in ("amod", "compound", "det")]
        symptomes.append(" ".join(mods + [token.text]))
    
    if token.dep_ == "conj" and token.head.lemma_ == "présente":
        mods = [child.text for child in token.children if child.dep_ in ("amod", "compound", "det")]
        symptomes.append(" ".join(mods + [token.text]))
symptomes

['une toux persistante', 'une fièvre']

In [70]:
traitements = []
for token in doc:
    if token.lemma_ == "prescrire" and token.pos_ == "VERB":
        for child in token.children:
            dose = [c.text for c in child.children if c.dep_ in ("amod", "nummod", "det")]
            traitements.append(" ".join([child.text] + dose))
traitements

["amoxicilline l' 500mg"]

## Partie 2 : Extraction de Relations 

In [71]:
def extraire_relations(text:str,nlp):
    doc = nlp(text)
    relations = []
    for token in doc:
        if token.pos_ == "VERB":
            sujet = None
            objet = None
            for child in token.children:
                if child.dep_ in ("nsubj", "nsubj:pass"):
                    sujet = child.text
                elif child.dep_ in ("obj", "obl:arg"):
                    objet = child.text
            if sujet and objet:
                relations.append((sujet, token.lemma_, objet))
    return relations


In [72]:
from analyse_medicale import extraire_relations

ModuleNotFoundError: No module named 'analyse_medicale'

In [ ]:
rs_text = extraire_relations("Le médecin arrête l'aspirine en raison de saignements.",nlp)
rs_text

[('médecin', 'arrêter', 'aspirine'), ('saignements', 'raison', 'arrêter')]

## Partie 3 : Analyse de Dépendances

In [ ]:
from spacy import displacy

disp_test = "Après analyse, le cardiologue recommande un scanner cardiaque immédiat."
doc=nlp(disp_test)

displacy.render(doc, style="dep", jupyter=True)

In [ ]:
from analyse_medicale import extraire_medicaments_posologie


In [ ]:
extraire_medicaments_posologie(nlp,texte)

[('amoxicilline', '500mg 3x/jour')]

In [74]:
phrases_test = [
    "Le patient n' a pas refuse l'anticoagulant malgré son AVC récent.",
    "Prescription : ibuprofène 400mg si douleur, maximum 3 comprimés/jour.",
    "Pas d'antibiothérapie pour cette infection virale.",
    "Le médecin arrête l'aspirine en raison de saignements.",
    "Le patient prend du paracétamol chaque matin.",
    "Le docteur ne prescrit pas d'antibiotiques.",
    "La patiente reçoit une perfusion à cause de la déshydratation.",
    "On suspend le traitement pour éviter les effets secondaires."
]

print("Extraction des relations :\n")
for sent in phrases_test:
    print(extraire_relations(sent,nlp))

Extraction des relations :

[('patient', 'refuser', 'anticoagulant')]
[]
[]
[('médecin', 'arrêter', 'aspirine')]
[('patient', 'prendre', 'paracétamol')]
[('docteur', 'prescrire', 'antibiotiques')]
[('patiente', 'recevoir', 'perfusion')]
[('On', 'suspendre', 'traitement')]
